In [1]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library.ansatzes import UCC
from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint
from qiskit.circuit import CircuitInstruction, instruction, Gate
from qiskit.quantum_info import PauliList, Pauli
from qiskit.quantum_info.operators.symplectic import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit.circuit.library import HamiltonianGate, PauliEvolutionGate
from qiskit_algorithms import TrotterQRTE, TimeEvolutionProblem
from qiskit.primitives import Estimator
from qiskit import transpile
from Paulihedral_v2.Paulihedral_new.benchmark.mypauli import *
from Paulihedral_v2.Paulihedral_new.parallel_bl import depth_oriented_scheduling, gate_count_oriented_scheduling
from Paulihedral_v2.Paulihedral_new.tools import *
from qiskit import transpile
import time
from Paulihedral_v2.Paulihedral_new import synthesis_FT

In [2]:
from qiskit_nature.second_q.circuit.library.ansatzes.utils import generate_fermionic_excitations
from Ternary_Tree.CircAlgorithm.UpUCCSDG import UpUCCSDG
from Ternary_Tree.OpenFermionSQ.Ternary_Tree_mapper import TernaryTreeMapper
from qiskit_nature.second_q.mappers import BravyiKitaevMapper

basis="6-311g"
names = 'Li 0 0 0; Li 0 0 '
active_orbitals = [0,1,2,3,4,5]
n = len(active_orbitals)
# active_orbitals = None

def init(dist=0.714):
    global ucc, fermionic_op
    ucc = UpUCCSDG(geometry=names + str(dist), basis=basis, active_orbitals=active_orbitals, num_electrons=(1,1))
    fermionic_op = ucc.mol.hamiltonian.second_q_op()
init()
al = ucc.get_alpha_excitations()
be = ucc.get_beta_excitations()
do = ucc.get_double_excitations()
# print(al)
# print(be)
# print(do)
def my_generation(**kwargs):
    global al,be,do
    ls1 = []
    ls2 = []
    for el in al:
        ls1.append(((el[1],), (el[0],)))
    for el in be:
        ls1.append(((el[1],), (el[0],)))
    for el in do:
        if el[0] > el[2]:
            ls2.append(((el[0],el[1]), (el[2],el[3])))
    # print(ls)
    print(ls1 + ls2)
    return ls1 + ls2
toto = ucc.get_parametrized_circuit()
# qubit_mapper = TernaryTreeMapper(ucc.tt)
# qubit_mapper = BravyiKitaevMapper()
qubit_mapper = JordanWignerMapper()
new_ucc = UCC(ucc.num_spatial_orbitals, (ucc.num_alpha, ucc.num_beta), excitations=my_generation, qubit_mapper=qubit_mapper)
# new_ucc = toto
# generate_fermionic_excitations(2, 3, (1, 1), generalized=True)

The following error was encountered during the attempted conversion of the 4-fold to 8-fold symmetric integrals: Expected a symmetric tensor.


{}
[((0,), (1,)), ((0,), (2,)), ((1,), (2,)), ((0,), (3,)), ((1,), (3,)), ((2,), (3,)), ((0,), (4,)), ((1,), (4,)), ((2,), (4,)), ((3,), (4,)), ((0,), (5,)), ((1,), (5,)), ((2,), (5,)), ((3,), (5,)), ((4,), (5,)), ((6,), (7,)), ((6,), (8,)), ((7,), (8,)), ((6,), (9,)), ((7,), (9,)), ((8,), (9,)), ((6,), (10,)), ((7,), (10,)), ((8,), (10,)), ((9,), (10,)), ((6,), (11,)), ((7,), (11,)), ((8,), (11,)), ((9,), (11,)), ((10,), (11,)), ((7, 1), (6, 0)), ((8, 2), (6, 0)), ((8, 2), (7, 1)), ((9, 3), (6, 0)), ((9, 3), (7, 1)), ((9, 3), (8, 2)), ((10, 4), (6, 0)), ((10, 4), (7, 1)), ((10, 4), (8, 2)), ((10, 4), (9, 3)), ((11, 5), (6, 0)), ((11, 5), (7, 1)), ((11, 5), (8, 2)), ((11, 5), (9, 3)), ((11, 5), (10, 4))]


In [3]:
parr = []
from numpy.random import shuffle
for gate in new_ucc.decompose(reps=2):
    parr.append([pauliString(gate.operation.name[-1-n*2:-1], 1.0)])
print(parr)
shuffle(parr)
ctime = time.time
nq = len(parr[0][0])
# length = nq//2 # `length' is a hyperparameter, and can be adjusted for best performance
t0 = ctime()
a1 = gate_count_oriented_scheduling(parr)
qc = synthesis_FT.block_opt_FT(a1)
qc2 = transpile(qc, basis_gates=['u3', 'cx'], optimization_level=3)
print_qc(qc2)

[[IIIIIIIIIIXY], [IIIIIIIIIIYX], [IIIIIIIIIXZY], [IIIIIIIIIYZX], [IIIIIIIIIXYI], [IIIIIIIIIYXI], [IIIIIIIIXZZY], [IIIIIIIIYZZX], [IIIIIIIIXZYI], [IIIIIIIIYZXI], [IIIIIIIIXYII], [IIIIIIIIYXII], [IIIIIIIXZZZY], [IIIIIIIYZZZX], [IIIIIIIXZZYI], [IIIIIIIYZZXI], [IIIIIIIXZYII], [IIIIIIIYZXII], [IIIIIIIXYIII], [IIIIIIIYXIII], [IIIIIIXZZZZY], [IIIIIIYZZZZX], [IIIIIIXZZZYI], [IIIIIIYZZZXI], [IIIIIIXZZYII], [IIIIIIYZZXII], [IIIIIIXZYIII], [IIIIIIYZXIII], [IIIIIIXYIIII], [IIIIIIYXIIII], [IIIIXYIIIIII], [IIIIYXIIIIII], [IIIXZYIIIIII], [IIIYZXIIIIII], [IIIXYIIIIIII], [IIIYXIIIIIII], [IIXZZYIIIIII], [IIYZZXIIIIII], [IIXZYIIIIIII], [IIYZXIIIIIII], [IIXYIIIIIIII], [IIYXIIIIIIII], [IXZZZYIIIIII], [IYZZZXIIIIII], [IXZZYIIIIIII], [IYZZXIIIIIII], [IXZYIIIIIIII], [IYZXIIIIIIII], [IXYIIIIIIIII], [IYXIIIIIIIII], [XZZZZYIIIIII], [YZZZZXIIIIII], [XZZZYIIIIIII], [YZZZXIIIIIII], [XZZYIIIIIIII], [YZZXIIIIIIII], [XZYIIIIIIIII], [YZXIIIIIIIII], [XYIIIIIIIIII], [YXIIIIIIIIII], [IIIIXYIIIIXX], [IIIIXXIIIIXY], [IIIIXY

(569, 713, 889)

In [4]:
from numpy.random import random
par = new_ucc.parameters
qc = new_ucc.assign_parameters({el: random() for el in par})
circ = transpile(qc,  optimization_level=3, basis_gates=['id', 'rz', 'rx','cx','h','s'])
circ = circ.decompose(reps=3)
# print(circ)
# print(transpile(qc.decompose(reps=2)))
# num = 0
# for inst in circ.data:
#     if len(inst.qubits) == 2:
#         num += 1
# print(f"num = {num}")
print(f"depth = ", circ.depth() )
print(circ.decompose(reps=3).count_ops())

depth =  1477
OrderedDict({'cx': 1223, 'u': 889})


In [5]:
N = [4,6,8,10,12,14]
num_temrs = [72, 180, 336, 540, 792, 1092, 1440]
cx_lex_jw = [204, 503, 948, 1525, 2236, 3084, 4068]
depth_lex_jw = [328, 633, 1516, 2424, 3568, 4918, 6484]
# (2236, 2754, 3568)
cx_lex_bk = [222, 772, 1484, 2995, 4631, 6530, 8388, 12441, 16202, 20472]
depth_lex_bk = [325, 1053, 1950, 3795, 5719, 8043, 10065, 14830, 19125, 24094]

cx_bk = [222, 772, 1484, 2995, 4631, 6530, 8388, 22813, 30081, 38219]
depth_bk = [325, 1053, 1950, 3795, 5719, 8043, 10065, 27273, 36066, 45799]

cx_thebest = [154, 382, 680, 1056, 1502, 2018]
depth_thebest = [95, 174, 211, 297, 334, 394]

In [6]:
# pl = []
# for gate in new_ucc.decompose(reps=2):
#     pl.append(Pauli(gate.operation.name[-1-n*2:-1]))

# print(len(pl))
# pl = PauliList(pl)

# pl = pl.sort()
# sp = SparsePauliOp(pl, coeffs=[0.23 for _ in pl])
# mc = QuantumCircuit(2*n)
# from numpy.random import random
# for el in pl:
#     mc.append(PauliEvolutionGate(el, random()),[i for i in range(2*n)])
# circ = mc.decompose()
# print(circ.count_ops())
# new_circ = transpile(circ, basis_gates=['u3', 'cx'], optimization_level=3)
# print(new_circ.decompose(reps=4).count_ops())
# print(new_circ.count_ops())
# print(new_circ.decompose(reps=4).depth())

[-1.8842314684076118, -1.8842431065487353, -1.8921159768871416, -1.8842431065487246, -1.892117630116096, 0.42779330137770133, 0.4997015141402404]
[-1.530699539615456, -1.725128919171379, -1.6390915181743695, -1.6345047089736497, -1.746487123243178, 0.4719970043431742, -0.6354287643477874]


[-8.983075763063907, -8.996330743493685, -8.996271447555635, -8.996176532608297, -8.996135113257905]
[-8.95233613357146, -8.924138096925946, -8.92893314227695, -8.955818944668815, -8.958489808114432]

# h_2

180
OrderedDict({'cx': 1640, 'h': 1200, 'sdg': 300, 's': 300, 'rz': 180})
OrderedDict({'u': 768, 'cx': 718})
OrderedDict({'u3': 768, 'cx': 718})
1079


In [6]:
data1 = [[-1.878062728950053, -1.8780188081815938, -1.8779901675978046, -1.8780159642789465, -1.8779956811670544], [-1.8780627289500536, -1.8768015312145414, -1.8777325112545247, -1.8772390031426858, -1.8774874816387057], [-1.8780627289500513, -1.874061317882318, -1.876016350870364, -1.8761609060646136, -1.8758787410988824], [-1.8780627289500536, -1.8724152757503918, -1.8729425830315711, -1.8718001637136166, -1.8723024748554327], [-1.8780627289363008, -1.871151782859928, -1.872317606404081, -1.8721297097765588, -1.8725776867360495], [-1.8780627230828373, -1.8690019660241903, -1.8702131010584788, -1.8691900222757418, -1.871109570207809], [-1.878062721358516, -1.8689382880390728, -1.8694669507179196, -1.8702339139427828, -1.8680705358587497], [-1.8780614098291741, -1.866619781804208, -1.8675933499227693, -1.8667145623791848, -1.8665058483376067], [-1.87806162378661, -1.867120983856301, -1.8679989021698389, -1.8680279266421806, -1.8672325125049916], [-1.8780576185606876, -1.866704112569608, -1.8665155280235823, -1.8662213514690917, -1.8681190616290946]]
data2 = [[-1.869660293527135, -1.8689728671060173, -1.870991607500812, -1.8702224114662025, -1.8751922729979011], [-1.870733981430148, -1.868267215055872, -1.8701314615829712, -1.8695891170935712, -1.8748355648934538], [-1.8707355311689313, -1.8673763878051042, -1.8690975378655497, -1.867122136923656, -1.872676560318483], [-1.8707340341143353, -1.8632018863336444, -1.8662486093638968, -1.8658444370737781, -1.8694828907245897], [-1.8707339147500832, -1.8624054629889528, -1.8632150710473772, -1.8640470494889179, -1.8692606068033353], [-1.870733920770873, -1.8596939004951227, -1.8638376412948232, -1.8626186086753802, -1.8667349300693798], [-1.8707337004816662, -1.8573304350139646, -1.8618285630048024, -1.8615732979497996, -1.8658303695718987], [-1.8707336255265734, -1.860345754677584, -1.8619103830208314, -1.8578864102458532, -1.865182743685979], [-1.8707336012499254, -1.8580980777587761, -1.8604187365614255, -1.8589788118569328, -1.865064174601412], [-1.8706655481191876, -1.8576453609155235, -1.8605823001267319, -1.8587190679632675, -1.8626109480160375]]
data3 = [[-1.7663167573228875, -1.8839249340464348, -1.8841457018597119, -1.8842078878196278, -1.8841037053938579], [-1.8266205395870276, -1.8834461563834815, -1.8835976630504392, -1.8834572013656488, -1.8837439064990211], [-1.8634688119590783, -1.8833487681276164, -1.8836777819747448, -1.8835071832402546, -1.8832646742954018], [-1.882349009744673, -1.8827533485854635, -1.8832602070447035, -1.8830066135120327, -1.8835279738744357], [-1.8826365350470045, -1.8822136265473361, -1.8821007444552371, -1.8820226235457442, -1.8823205459080998], [-1.883173942084247, -1.8815584585689233, -1.8810016535313632, -1.8807876669778352, -1.8819576661590212], [-1.8841318295178446, -1.8806471155505111, -1.8815885831060002, -1.88043302478314, -1.8808431290605798], [-1.8814344706106136, -1.8811538946205346, -1.879788263919544, -1.880198427771074, -1.8801951524260752], [-1.883083870982141, -1.8802452099471285, -1.879945072777876, -1.8790356230171044, -1.8798039437170648], [-1.8829353155087392, -1.8800653136625085, -1.8804572738038652, -1.8785051600387113, -1.87916104824124]]
data4 = [[-1.8632015543091627, -1.8634428803710656], [-1.863335475622211, -1.8632447120051099], [-1.863338556341452, -1.8633258152387515], [-1.8633064092958442, -1.8632054508151843], [-1.8634605566567646, -1.863316500251401], [-1.8633587165644065, -1.8632676529753516], [-1.863315788120613, -1.8634660389658841], [-1.8634001799953765, -1.8633926527506013], [-1.863403747341637, -1.863288089757124], [-1.863393357377564, -1.8631784635287356]]
# for obj in data1:
#     print(obj)
# print("-----------")
# for obj in data2:
#     print(obj)
print("-----------")

for obj in data3:
    print(obj)
    
print('-----------')

for obj in data4:
    print(obj)



-----------
[-1.7663167573228875, -1.8839249340464348, -1.8841457018597119, -1.8842078878196278, -1.8841037053938579]
[-1.8266205395870276, -1.8834461563834815, -1.8835976630504392, -1.8834572013656488, -1.8837439064990211]
[-1.8634688119590783, -1.8833487681276164, -1.8836777819747448, -1.8835071832402546, -1.8832646742954018]
[-1.882349009744673, -1.8827533485854635, -1.8832602070447035, -1.8830066135120327, -1.8835279738744357]
[-1.8826365350470045, -1.8822136265473361, -1.8821007444552371, -1.8820226235457442, -1.8823205459080998]
[-1.883173942084247, -1.8815584585689233, -1.8810016535313632, -1.8807876669778352, -1.8819576661590212]
[-1.8841318295178446, -1.8806471155505111, -1.8815885831060002, -1.88043302478314, -1.8808431290605798]
[-1.8814344706106136, -1.8811538946205346, -1.879788263919544, -1.880198427771074, -1.8801951524260752]
[-1.883083870982141, -1.8802452099471285, -1.879945072777876, -1.8790356230171044, -1.8798039437170648]
[-1.8829353155087392, -1.8800653136625085,

In [1]:
import numpy as np

a = np.array([0.0760115602532588, 2.8146550012274702e-05, 0.34095812418868954, -0.03749392681860998, -2.2162879711760454e-05, -0.15254938158533515, 0.008696352092301251, 0.07605927421688939, -0.0056892209935557195, -0.043613956505113155, -2.9088354756806684e-05, -0.2764064437196367, -0.17948778573366828, 0.001833846784728066, -0.005807164177023991, 0.029821868934014728, 3.740948975144749e-05, 0.19210059935589763, 0.0005687079853326252, 5.595659451305561e-07, -0.33413959765139006, -0.008833961982741211, 1.6987905570859238e-05, 0.21314221325429217, -0.023232530282582813, -0.000713657211555915, -0.0036516304767531226, -0.08173382112968, -2.7378634292466043e-07, 0.2676678595988892, 0.1615995008068559, 0.006436166951252286, 0.0006685449050682626, 0.010866044259779505, -3.162445574994113e-05, -0.26214564786995126])
b = np.array([0.1299775175804247, -5.337794601141431e-06, 1.1857947327840195, -0.07949088879974482, -0.00019336486961418965, 0.3746728406014784, 0.38688389300310805, -0.18249160163048675, 0.03160178677595464, 0.022221128209917132, 0.04463411515522797, 0.002572223173160859, -0.0165961328587215, 0.7124080881770755, -0.778799577472838, -0.08914103134639072, -1.2047019416516099e-05, 0.012420712357562829, -0.04054693861093698, -5.5535102851208625e-05, -1.558154614757748, 7.587060332957359e-05, -0.0008403185043895874, -0.9027959476758446, -0.15418794484005988, 0.0021890566246680326, 0.01506070993359596, -0.011297031123496684, 2.9920005924406837e-05, -0.039022026908700734, -0.04956765318297777, 0.06947573105945043, -0.15998529005176965, -9.331757387943655e-05, 4.5986432398653886e-05, -0.01233091799911377])
print(a - b)

[-5.39659573e-02  3.34843446e-05 -8.44836609e-01  4.19969620e-02
  1.71201990e-04 -5.27222222e-01 -3.78187541e-01  2.58550876e-01
 -3.72910078e-02 -6.58350847e-02 -4.46632035e-02 -2.78978667e-01
 -1.62891653e-01 -7.10574241e-01  7.72992413e-01  1.18962900e-01
  4.94565092e-05  1.79679887e-01  4.11156466e-02  5.60946688e-05
  1.22401502e+00 -8.90983259e-03  8.57306410e-04  1.11593816e+00
  1.30955415e-01 -2.90271384e-03 -1.87123404e-02 -7.04367900e-02
 -3.01937923e-05  3.06689887e-01  2.11167154e-01 -6.30395641e-02
  1.60653835e-01  1.09593618e-02 -7.76108881e-05 -2.49814730e-01]


In [34]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

from numpy import pi

theta = pi/6
# theta = 0
qc = QuantumCircuit(4)
qc.cx(1,0)
qc.cx(2,3)
qc.ry(pi/2,2)
qc.cx(2,1)
qc.ry(-theta,1)
qc.ry(-theta, 2)
qc.cx(0,1)
qc.cx(3,2)
qc.ry(-theta,1)
qc.ry(-theta, 2)
qc.cx(0,2)
qc.cx(3,1)
qc.ry(-theta,1)
qc.ry(-theta, 2)
qc.cx(0,1)
qc.cx(3,2)
qc.ry(-theta,1)
qc.ry(-theta, 2)
qc.cx(0,2)
qc.cx(3,1)
qc.cx(2,1)
qc.ry(-pi/2, 2)
qc.cx(1,0)
qc.cx(2,3)
print(qc)
ls = [0 for i in range(16)]
ls[3] = 1
# ls[12] = 1/2
state = Statevector(ls)
state.draw("latex")
state = state.evolve(qc)
state.draw("latex")

     ┌───┐                                                                   »
q_0: ┤ X ├──────────────────────────────■────────────────■───────────────────»
     └─┬─┘           ┌───┐┌──────────┐┌─┴─┐┌──────────┐  │  ┌───┐┌──────────┐»
q_1: ──■─────────────┤ X ├┤ Ry(-π/6) ├┤ X ├┤ Ry(-π/6) ├──┼──┤ X ├┤ Ry(-π/6) ├»
          ┌─────────┐└─┬─┘├──────────┤├───┤├──────────┤┌─┴─┐└─┬─┘├──────────┤»
q_2: ──■──┤ Ry(π/2) ├──■──┤ Ry(-π/6) ├┤ X ├┤ Ry(-π/6) ├┤ X ├──┼──┤ Ry(-π/6) ├»
     ┌─┴─┐└─────────┘     └──────────┘└─┬─┘└──────────┘└───┘  │  └──────────┘»
q_3: ┤ X ├──────────────────────────────■─────────────────────■──────────────»
     └───┘                                                                   »
«                                        ┌───┐         
«q_0: ──■────────────────■───────────────┤ X ├─────────
«     ┌─┴─┐┌──────────┐  │  ┌───┐┌───┐   └─┬─┘         
«q_1: ┤ X ├┤ Ry(-π/6) ├──┼──┤ X ├┤ X ├─────■───────────
«     ├───┤├──────────┤┌─┴─┐└─┬─┘└─┬─┘┌──────────┐     
«q_2: ┤ X

<IPython.core.display.Latex object>